In [1]:
import pandas as pd
import jieba

In [2]:
TRAIN_PATH = "/home/abner/PycharmProjects/NLP_Pro_01/database/AutoMaster_TrainSet.csv"
TEST_PATH = "/home/abner/PycharmProjects/NLP_Pro_01/database/AutoMaster_TestSet.csv"
STOP_WORDS = '/home/abner/PycharmProjects/NLP_Pro_01/database/StopWords.txt'

In [66]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

In [67]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82943 entries, 0 to 82942
Data columns (total 6 columns):
QID         82943 non-null object
Brand       81642 non-null object
Model       81642 non-null object
Question    82943 non-null object
Dialogue    82941 non-null object
Report      82873 non-null object
dtypes: object(6)
memory usage: 3.8+ MB


In [68]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


In [69]:
train_df.dropna(subset=['Question', 'Dialogue', 'Report'], how='any', inplace=True)
test_df.dropna(subset=['Question', 'Dialogue'], how='any', inplace=True)


In [12]:
import re

def clean_sentence(sentence):
    '''
    特殊符号去除
    使用正则表达式去除无用的符号、词语
    '''
    if isinstance(sentence, str):
        return re.sub(
            r'[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】“”！，。？、~@#￥%……&*（）]+|车主说|技师说|语音|图片|你好|您好',
            '', sentence)
    else:
        return ''

## 创建自己的自定义词典，保证一些专用名词不会被切分，例如品牌型号

In [70]:
brand_words = set(train_df['Brand'])
model_words = set(train_df['Model'])
sp_words = brand_words | model_words

In [39]:
with open('database/user_dict.txt', 'w') as f:
    for i in sp_words:
        i = str(i)
        f.write(i+'\n')

In [42]:
user_dict='database/user_dict.txt'
# 载入自定义词典
jieba.load_userdict(user_dict)

In [43]:
stop = []
with open(STOP_WORDS, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        stop.append(line.strip())

In [59]:
def cut_words(sentences, stop=stop):
    # 清除无用词
    sentence = clean_sentence(sentences)
    # 切词，默认精确模式，全模式cut参数cut_all=True
    words = jieba.cut(sentence)
    # 过滤停用词
    words = [w for w in words if w not in stop]
    return ' '.join(words)

In [52]:
cut_words('创建自己的自定义词典，保证一些专用名词不会被切分，例如品牌型号')

['创建', '自定义', '词典', '保证', '专用名词', '不会', '切分', '品牌', '型号']

In [60]:
# 批处理
def cut_data_frame(df):
    '''
    数据集批量处理方法
    :param df: 数据集
    :return:处理好的数据集
    '''
    # 批量预处理 训练集和测试集
    for col_name in ['Brand', 'Model', 'Question', 'Dialogue']:
        df[col_name] = df[col_name].apply(cut_words)

    if 'Report' in df.columns:
        # 训练集 Report 预处理
        df['Report'] = df['Report'].apply(cut_words)
    return df

## 多核加速

In [54]:
import numpy as np
from multiprocessing import cpu_count, Pool

In [55]:
# cpu 数量
cores = cpu_count()
# 分块个数
partitions = cores 

In [56]:
def parallelize(df, func):
    """
    多核并行处理模块
    :param df: DataFrame数据
    :param func: 预处理函数 
    :return: 处理后的数据
    """
    # 数据切分
    data_split = np.array_split(df, partitions)
    # 进程池
    pool = Pool(cores)
    # 数据分发 合并
    data = pd.concat(pool.map(func, data_split))
    # 关闭进程池
    pool.close()
    # 执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    pool.join()
    return data

In [71]:
%%time
# 多线程处理预处理
train_df = parallelize(train_df, cut_data_frame);
test_df = parallelize(test_df, cut_data_frame);

CPU times: user 462 ms, sys: 385 ms, total: 847 ms
Wall time: 43.7 s


In [72]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机 重 助力 泵 方向机 都 换,新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问...,随时 联系
1,Q2,奔驰,奔驰M级,奔驰 ML500 排气 凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马X1 进口,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先 挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,Jeep,牧马人,30V6 发动机 号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰C级,2012 款 奔驰 c180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场价 钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


In [74]:
train_df.to_csv('database/train_seg_data.csv',index=None,header=True)
test_df.to_csv('database/test_seg_data.csv',index=None,header=True)